# Costruisci un algoritmo di raccomandazione from scratch

## Parsing del dataset Netflix

In questa fase, importiamo e interpretiamo il file CSV contenente i titoli di Netflix, strutturando ogni riga come un dizionario Python.

L'uso di [csv.DictReader](https://docs.python.org/3/library/csv.html#csv.DictReader) consente di leggere direttamente i valori associandoli alle intestazioni delle colonne, facilitando così l'accesso ai campi tramite nomi simbolici.

Durante il parsing:

- I campi testuali come `cast` e `listed_in` vengono convertiti in liste.
- I valori mancanti vengono gestiti e ripuliti con `.strip()` e controlli condizionali.
- Il campo `release_year` viene convertito in intero solo se rappresenta un numero valido.

Questa standardizzazione iniziale è essenziale per rendere i dati utilizzabili nei successivi step di analisi e raccomandazione.


In [1]:
# Step 0

import csv
import json 

# INPUT: path (string) - percorso del file CSV
# OUTPUT: lista di dizionari, uno per ogni film/serie
# SCOPO: carica e pulisce i dati da un file CSV Netflix
def carica_dati_netflix(path):
    dataset = []
    return dataset

# Esempio d'uso
# dati = carica_dati_netflix('data/netflix_titles.csv')
# print(json.dumps(dati[7659], indent=2))


## Raccomandazioni per genere (filtraggio semplice)

Queste due funzioni implementano una forma elementare di **sistema di raccomandazione basato su contenuto**, sfruttando il campo `listed_in` del dataset, che rappresenta i generi associati a ciascun titolo.

### `raccomanda_per_genere`

Filtra i titoli per singolo genere (es. "Drama"). Il confronto è case-insensitive e si basa su un'equivalenza diretta tra il genere richiesto e quelli associati al titolo. I risultati sono ordinati per anno di uscita in ordine decrescente.

### `raccomanda_per_generi`

Supporta la richiesta di più generi contemporaneamente (es. "Drama, Thriller"). L'algoritmo include tutti i titoli che contengono **almeno uno** dei generi richiesti.

Queste funzioni non considerano ancora alcuna misura di similarità, distanza o vettorizzazione. Sono esempi di **filtraggio deterministico**, utile come baseline per confrontare approcci successivi più sofisticati.


In [2]:
# Step 1

# INPUT: 
#   dati (list of dict) - dataset dei titoli Netflix, ogni dict rappresenta un contenuto
#   genere_input (str) - nome del genere da cercare (case-insensitive)
# OUTPUT: 
#   None - la funzione stampa i risultati a schermo
# SCOPO: 
#   stampa i titoli appartenenti al genere richiesto, ordinati per anno decrescente
def raccomanda_per_genere(dati, genere_input):
    pass

# INPUT: 
#   dati (list of dict) - dataset dei titoli Netflix, ogni dict rappresenta un contenuto
#   input_generi (str) - stringa con uno o più generi separati da virgole (es. "Drama, Thriller")
# OUTPUT: 
#   None - la funzione stampa i risultati a schermo
# SCOPO: 
#   stampa i titoli che appartengono ad almeno uno dei generi richiesti, ordinati per anno decrescente
def raccomanda_per_generi(dati, input_generi):
    pass

# ESEMPIO USO
# generi = input("Inserisci uno o più generi separati da virgola (es. Drama, Thriller): ")
# raccomanda_per_generi(dati, generi)


# ESEMPIO D'USO
# genere = input("Inserisci un genere (es. Dramas, Thrillers): ")
# raccomanda_per_genere(dati, genere)


## Raccomandazione basata su similarità nel feature space

A questo punto, passiamo da un filtraggio simbolico a una rappresentazione quantitativa. Ogni film viene proiettato in uno **spazio delle feature** definito dai generi presenti nel dataset.

### One Hot Encoding binario

Si costruisce un **vocabolario** di tutti i generi presenti (`costruisci_vocabolario_generi`) e ogni film viene rappresentato da un vettore binario (`vettore_generi`) della stessa lunghezza del vocabolario:

- 1 se il film appartiene al genere in quella posizione
- 0 altrimenti

Questa codifica è un'applicazione diretta del **One Hot Encoding** per feature categoriali multivalore.

### Similarità del coseno

La **similarità tra vettori** è calcolata tramite la misura del coseno:

$$
\text{sim}(\vec{A}, \vec{B}) = \frac{\vec{A} \cdot \vec{B}}{\|\vec{A}\| \cdot \|\vec{B}\|}
$$

Dove:
- $\vec{A} \cdot \vec{B}$ è il prodotto scalare tra i due vettori
- $\|\vec{A}\|$ e $\|\vec{B}\|$ sono le loro norme euclidee

Questa metrica restituisce un valore tra 0 e 1 che indica quanto due vettori sono **orientati nella stessa direzione**. Non dipende dalla magnitudine, rendendola adatta per vettori binari.

### Prossimità semantica

Il concetto di **prossimità** viene dunque tradotto in termini geometrici: due film sono considerati simili se occupano posizioni vicine (in termini angolari) nello spazio generato dai generi.

![Similarità del coseno tra vettori](asset/cosine-similarity-vectors.original.jpg)

La funzione `raccomanda_simili` restituisce i top-N film più vicini al titolo di input secondo questa metrica.


In [3]:
import math

# INPUT: 
#   dati (list of dict) - dataset dei titoli Netflix, ogni dict contiene una lista di generi in 'listed_in'
# OUTPUT: 
#   list of str - vocabolario ordinato di tutti i generi unici presenti nel dataset (in lowercase)
# SCOPO: 
#   costruire una lista di generi unici da usare come base per la vettorizzazione
def costruisci_vocabolario_generi(dati):
    pass

# INPUT: 
#   record (dict) - un singolo titolo con campo 'listed_in' (lista di generi)
#   vocabolario (list of str) - lista ordinata di tutti i generi possibili
# OUTPUT: 
#   list of int - vettore binario che indica la presenza (1) o assenza (0) di ciascun genere
# SCOPO: 
#   rappresentare i generi di un titolo come vettore binario basato sul vocabolario
def vettore_generi(record, vocabolario):
    pass

# INPUT: 
#   v1 (list of float) - primo vettore numerico
#   v2 (list of float) - secondo vettore numerico
# OUTPUT: 
#   float - similarità del coseno tra i due vettori (valore tra 0 e 1)
# SCOPO: 
#   calcolare quanto due vettori sono orientati nella stessa direzione nello spazio
def similarita_coseno(v1, v2):
    pass

# INPUT: 
#   dati (list of dict) - dataset dei titoli Netflix
#   titolo_input (str) - titolo da cercare (case-insensitive)
# OUTPUT: 
#   dict or None - record corrispondente al titolo, oppure None se non trovato
# SCOPO: 
#   trovare e restituire il dizionario di un titolo dato il nome, ignorando maiuscole/minuscole
def trova_record_per_titolo(dati, titolo_input):
    pass

# INPUT: 
#   dati (list of dict) - dataset dei titoli Netflix
#   titolo_input (str) - titolo da usare come riferimento (case-insensitive)
#   top_n (int) - numero di titoli simili da restituire (default: 10)
# OUTPUT: 
#   None - la funzione stampa i titoli simili a schermo
# SCOPO: 
#   confrontare un titolo con tutti gli altri usando la similarità sui generi e stampare i top-N più simili
def raccomanda_simili(dati, titolo_input, top_n=10):
    pass

# print("GENERI: ", costruisci_vocabolario_generi(dati), "\n")
# print(dati[7659]['title'], '->', dati[7659]['listed_in'])
# print("VETTORE:", vettore_generi(dati[7659], costruisci_vocabolario_generi(dati)), "\n")
# print("Cos Similarity: ", similarita_coseno([0, 1, 0], [1, 0, 0]), "\n")

In [4]:
# titolo = input("Inserisci il titolo di un film o serie: ")
# raccomanda_simili(dati, titolo)

## Espansione dello spazio vettoriale con feature categoriali multiple

Finora lo spazio delle feature era limitato ai generi. In questa fase, estendiamo la rappresentazione vettoriale di ogni titolo includendo altre feature categoriali discrete:

- `rating` (es. TV-MA, PG, etc.)
- `type` (Movie o TV Show)
- `director` (filtrando solo i **top-N più frequenti**)

### Approccio

- Le feature vengono one-hot encodate esattamente come fatto per i generi.
- I valori di `rating` e `type` sono direttamente binarizzati.
- Per `director`, si evita la codifica completa (troppo dispersiva) e si selezionano solo i più frequenti tramite [`Counter`](https://docs.python.org/3/library/collections.html#collections.Counter).

La funzione `vettore_completo` concatena tutti i sottovettori binari in un unico vettore ad alta dimensionalità che rappresenta ogni contenuto in modo più ricco.

### Similarità estesa

La funzione `raccomanda_simili_esteso` sfrutta questa rappresentazione multidimensionale per calcolare la **similarità del coseno** su uno spazio vettoriale più articolato. Il sistema è ora in grado di cogliere affinità non solo tematiche (genere), ma anche stilistiche (regia), strutturali (tipo) e di target (rating).

Questo rappresenta un primo passo concreto verso un sistema **ibrido basato su contenuto**, più sofisticato del semplice filtraggio su un attributo.

## Uso di Counter

```python
from collections import Counter

nomi = ['Luca', 'Anna', 'Luca', 'Marco', 'Anna', 'Luca']
conteggio = Counter(nomi)
print(conteggio)

# output
Counter({'Luca': 3, 'Anna': 2, 'Marco': 1})

```

Versione senza Counter

```python
director_counter = {}

for r in dati:
    if r['director']:
        nome = r['director'].strip()
        if nome not in director_counter:
            director_counter[nome] = 1
        else:
            director_counter[nome] += 1

```

In [5]:
from collections import Counter

# INPUT: 
#   dati (list of dict) - dataset dei titoli Netflix
#   top_n_directors (int) - numero massimo di registi da includere (default: 20)
# OUTPUT: 
#   tuple (list of str, list of str, list of str) - vocabolario ordinato dei rating, tipi e top-N registi più frequenti
# SCOPO: 
#   estrarre i valori unici per rating e tipo, e i registi più frequenti per la codifica delle feature strutturate
def costruisci_vocabolari_categorici(dati, top_n_directors=20):
    pass


# INPUT: 
#   record (dict) - un titolo con campi 'listed_in', 'rating', 'type', 'director'
#   voc_generi (list of str) - vocabolario dei generi
#   voc_rating (list of str) - vocabolario dei rating
#   voc_type (list of str) - vocabolario dei tipi (Movie, TV Show)
#   voc_directors (list of str) - lista dei registi più frequenti
# OUTPUT: 
#   list of int - vettore binario concatenato che rappresenta tutte le feature strutturate del titolo
# SCOPO: 
#   costruire la rappresentazione vettoriale completa di un titolo su base categoriale
def vettore_completo(record, voc_generi, voc_rating, voc_type, voc_directors):
    pass

# INPUT: 
#   dati (list of dict) - dataset dei titoli Netflix
#   titolo_input (str) - titolo da usare come riferimento (case-insensitive)
#   top_n (int) - numero di titoli simili da stampare (default: 5)
# OUTPUT: 
#   None - la funzione stampa i risultati a schermo
# SCOPO: 
#   confrontare un titolo con tutti gli altri usando feature strutturate (generi, rating, tipo, regista) e stampare i top-N più simili
def raccomanda_simili_esteso(dati, titolo_input, top_n=5):
    pass
    

In [6]:
# titolo = input("Inserisci il titolo di un film o serie: ")
# raccomanda_simili_esteso(dati, titolo)

## Raccomandazione tramite analisi testuale: descrizioni e TF-IDF

A differenza delle feature categoriali precedenti, il campo `description` è testuale e necessita di un trattamento diverso. Usiamo il modello **TF-IDF** (Term Frequency – Inverse Document Frequency) per rappresentare ogni descrizione come un vettore sparso che evidenzia i termini più rilevanti.

### 1. Tokenizzazione e normalizzazione

La funzione `tokenizza_testo` converte la descrizione in una lista di token, rimuovendo punteggiatura e portando tutto in minuscolo. Questo step è cruciale per costruire un vocabolario consistente.

### 2. Term Frequency (TF)

Per ogni film, viene calcolata la **frequenza relativa** di ciascuna parola rispetto al totale delle parole nella descrizione:

$$
\text{TF}(t, d) = \frac{f_{t,d}}{\sum_{t'} f_{t',d}}
$$

### 3. Inverse Document Frequency (IDF)

L’IDF penalizza i termini troppo comuni nell’intero corpus, dando maggior peso a parole distintive:

$$
\text{IDF}(t) = \log \left( \frac{N}{1 + \text{df}(t)} \right)
$$

dove:
- $N$ è il numero totale di documenti
- $\text{df}(t)$ è il numero di documenti in cui il termine $t$ appare

### 4. Vettori TF-IDF

Ogni descrizione viene rappresentata come un dizionario sparso:

$$
\text{TF-IDF}(t, d) = \text{TF}(t, d) \cdot \text{IDF}(t)
$$

Questo consente di rappresentare contenuti con lunghezze diverse in uno spazio comune, evidenziando le parole semanticamente rilevanti.

### 5. Similarità del coseno su vettori sparsi

La similarità viene calcolata solo sulle parole in comune tra due descrizioni, evitando la costruzione esplicita di vettori densi di dimensione pari al vocabolario completo:

$$
\cos(\theta) = \frac{\sum_{i} A_i B_i}{\|A\| \cdot \|B\|}
$$

### 6. Raccomandazione semantica

`raccomanda_simili_descrizione` confronta la descrizione di un titolo con tutte le altre e restituisce i titoli semanticamente più vicini, sfruttando esclusivamente la rappresentazione testuale.

Questo approccio consente di cogliere **affinità semantiche latenti** non esplicitamente codificate nelle feature strutturate.


## Esempio manuale su vocabolario ridotto

Per chiarire il funzionamento del modello TF-IDF, consideriamo un sottoinsieme semplificato di tre descrizioni fittizie:

1. "A cat sits on the mat"
2. "The dog lies on the mat"
3. "A cat and a dog play together"

Supponiamo che il vocabolario risultante dalla tokenizzazione sia:

```python
["cat", "dog", "mat", "on", "the", "sits", "lies", "play", "together"]
```

### 🔹 Calcolo TF

Per la frase 1:

Totale parole = 6  
Parole: ["a", "cat", "sits", "on", "the", "mat"]

Dopo rimozione di stopword come "a", rimangono:

- TF("cat") = 1/6
- TF("sits") = 1/6
- TF("on") = 1/6
- TF("the") = 1/6
- TF("mat") = 1/6

### 🔹 Calcolo IDF

Numero documenti $N = 3$

| Termine   | Document Frequency (DF) | IDF                        |
|-----------|--------------------------|----------------------------|
| "cat"     | 2                        | $\log(3 / (1 + 2)) = 0$    |
| "sits"    | 1                        | $\log(3 / 2) \approx 0.405$|
| "mat"     | 2                        | $\log(3 / 3) = 0$          |
| "on"      | 2                        | $\log(3 / 3) = 0$          |
| "the"     | 3                        | $\log(3 / 4) < 0$          |

### 🔹 Costruzione del vettore TF-IDF

Moltiplichiamo TF per IDF per ciascun termine. I termini assenti non sono inclusi:

| Termine   | TF    | IDF     | TF-IDF    |
|-----------|-------|---------|-----------|
| "cat"     | 1/6   | 0.000   | 0.000     |
| "sits"    | 1/6   | 0.405   | 0.0675    |
| "mat"     | 1/6   | 0.000   | 0.000     |
| "on"      | 1/6   | 0.000   | 0.000     |
| "the"     | 1/6   | < 0     | 0.000     |

### 🔹 Vettore finale

Nel vocabolario completo di 9 termini, la frase 1 sarà rappresentata così:

**Forma densa (posizionale):**

```python
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0675, 0.0, 0.0, 0.0]
```


**Forma sparsa (più efficiente):**

```python
{"sits": 0.0675}
```

### Interpretazione

Termini comuni e distribuiti su molti documenti, come "cat", "mat" e "the", ricevono un peso vicino a zero: il modello li riconosce come poco informativi. Al contrario, termini più rari e distintivi come "sits" ottengono un peso maggiore nel vettore TF-IDF.

Questa rappresentazione enfatizza la specificità lessicale di ciascun documento, consentendo al sistema di raccomandazione di identificare affinità semantiche non evidenti nelle feature strutturate.



In [7]:
# INPUT: 
#   testo (str) - stringa da tokenizzare
# OUTPUT: 
#   list of str - lista di parole ottenute rimuovendo punteggiatura e convertendo in minuscolo
# SCOPO: 
#   normalizzare e suddividere un testo in token puliti per analisi testuale
def tokenizza_testo(testo):
    pass

# INPUT: 
#   dataset (list of dict) - ciascun dict rappresenta un titolo con una descrizione testuale
# OUTPUT: 
#   list of dict - una lista in cui ogni elemento è un dizionario {parola: frequenza relativa} per una descrizione
# SCOPO: 
#   calcolare la frequenza relativa (TF) delle parole in ciascuna descrizione del dataset
def calcola_tf(dataset):
    pass

# INPUT: 
#   tf_per_film (list of dict) - lista di dizionari contenenti le TF per ciascun titolo
# OUTPUT: 
#   dict - dizionario {parola: IDF}, con peso inverso alla frequenza nei documenti
# SCOPO: 
#   calcolare l'inverse document frequency per ogni parola nel corpus
def calcola_idf(tf_per_film):
    pass

# INPUT: 
#   tf_per_film (list of dict) - lista di TF per ciascuna descrizione
#   idf (dict) - dizionario {parola: IDF} calcolato sull'intero corpus
# OUTPUT: 
#   list of dict - lista di vettori TF-IDF sparsi per ciascun titolo
# SCOPO: 
#   calcolare il vettore TF-IDF per ogni descrizione combinando TF locale e IDF globale
def calcola_tfidf(tf_per_film, idf):
    pass

# INPUT: 
#   v1 (dict), v2 (dict) - due vettori TF-IDF sparsi {parola: peso}
# OUTPUT: 
#   float - similarità del coseno tra i due vettori (valore tra 0 e 1)
# SCOPO: 
#   calcolare la similarità del coseno tra vettori rappresentati come dizionari sparsi
def similarita_coseno_sparsa(v1, v2):
    pass

# INPUT: 
#   dataset (list of dict) - dataset dei titoli Netflix
#   vettori_tfidf (list of dict) - vettori TF-IDF sparsi associati alle descrizioni
#   titolo_input (str) - titolo di riferimento (case-insensitive)
#   top_n (int) - numero di titoli simili da mostrare (default: 5)
# OUTPUT: 
#   None - la funzione stampa i risultati a schermo
# SCOPO: 
#   trovare e stampare i top-N titoli con descrizioni più simili al titolo dato usando TF-IDF e similarità del coseno
def raccomanda_simili_descrizione(dataset, vettori_tfidf, titolo_input, top_n=5):
    pass

In [8]:
# tf = calcola_tf(dati)
# idf = calcola_idf(tf)
# tfidf = calcola_tfidf(tf, idf)

# print(dati[7659]['description'])
# print(json.dumps(tf[7659], indent=4))
# print(f"Term Frequency for 'his' -> {tf[7659]['his']}")

In [9]:
# titolo = input("Inserisci un titolo: ")
# print(once['description'])
# raccomanda_simili_descrizione(dati, tfidf, titolo)

## Raccomandazione ibrida: contenuto testuale + strutturale

Questa fase integra due rappresentazioni distinte di ciascun titolo:

- Un **vettore sparso TF-IDF** basato sulla descrizione testuale
- Un **vettore strutturato** binario costruito su generi, tipo, rating e regista

### Fusione dei segnali: similarità pesata

La funzione `similarita_pesata` calcola una **combinazione lineare** tra due misure di similarità del coseno:

$$
\text{sim}_{\text{ibrida}} = \alpha \cdot \text{sim}_{\text{TFIDF}} + (1 - \alpha) \cdot \text{sim}_{\text{struct}}
$$

dove:
- $\text{sim}_{\text{TFIDF}}$ è la similarità del coseno tra descrizioni testuali
- $\text{sim}_{\text{struct}}$ è la similarità del coseno tra vettori categoriali (generi, rating, ecc.)
- $\alpha$ è il **peso assegnato al contenuto testuale**, tipicamente $\alpha = 0.7$

### Algoritmo

La funzione `raccomanda_ibrido`:

1. Recupera sia la descrizione vettorializzata (`v_tfidf`) sia la rappresentazione strutturale (`v_struct`) del titolo richiesto
2. Confronta queste due rappresentazioni con tutti gli altri titoli del dataset
3. Calcola la similarità pesata
4. Restituisce i titoli con la similarità ibrida più alta

### Vantaggi del modello ibrido

- **Complementarietà semantica**: le descrizioni testuali catturano significati che non emergono da generi o classificazioni.
- **Robustezza strutturale**: i metadati strutturati compensano descrizioni brevi o poco informative.
- Il parametro $\alpha$ consente di **regolare dinamicamente** il bilanciamento tra le due fonti di informazione.

Questo approccio è una forma semplificata ma efficace di **content-based hybrid recommendation**.


In [10]:
# INPUT: 
#   v_tfidf1, v_tfidf2 (dict) - vettori TF-IDF sparsi dei due titoli
#   v_struct1, v_struct2 (list of int) - vettori strutturali binari dei due titoli
#   peso_tfidf (float) - peso assegnato alla similarità TF-IDF (default: 0.7)
# OUTPUT: 
#   float - similarità pesata combinando testo e struttura
# SCOPO: 
#   calcolare una similarità ibrida tra due titoli combinando descrizione testuale e feature strutturate
def similarita_pesata(v_tfidf1, v_struct1, v_tfidf2, v_struct2, peso_tfidf=0.7):
    pass

# INPUT: 
#   dati (list of dict) - dataset dei titoli Netflix
#   vettori_tfidf (list of dict) - vettori TF-IDF sparsi associati alle descrizioni
#   titolo_input (str) - titolo di riferimento (case-insensitive)
#   peso_tfidf (float) - peso assegnato alla componente testuale (default: 0.7)
#   top_n (int) - numero di titoli simili da mostrare (default: 5)
# OUTPUT: 
#   None - la funzione stampa i risultati a schermo
# SCOPO: 
#   raccomandare i top-N titoli più simili combinando descrizione testuale (TF-IDF) e metadati strutturali
def raccomanda_ibrido(dati, vettori_tfidf, titolo_input, peso_tfidf=0.7, top_n=5):
    pass


In [11]:
# titolo = input("Inserisci il titolo di un film o serie: ")
# raccomanda_ibrido(dati, tfidf, titolo, peso_tfidf=0.4)

## Raccomandazione semantica con Sentence Embeddings

In questa fase il sistema utilizza rappresentazioni dense delle descrizioni testuali, generate da modelli di tipo Transformer addestrati per catturare la semantica delle frasi. Ogni descrizione viene convertita in un vettore numerico ad alta dimensionalità che riflette il significato latente del testo, non solo le parole che lo compongono.

### Embedding semantici

Utilizzando un modello pre-addestrato come "all-distilroberta-v1", ogni descrizione viene proiettata in uno spazio vettoriale in cui frasi simili (anche se lessicalmente diverse) risultano vicine. Questi modelli sono stati ottimizzati per produrre embedding confrontabili tramite misure geometriche come la similarità del coseno.

A differenza del TF-IDF, che si basa su frequenze e presenze di termini, gli embeddings sono in grado di cogliere:

- sinonimie e parafrasi
- relazioni concettuali non esplicite
- contesti sintattici

Ad esempio, frasi come “Two kids are lost in space” e “A pair of children explore the galaxy” risultano semanticamente simili, pur non condividendo parole chiave.

### Raccomandazione su base semantica

Una volta ottenuti gli embedding delle descrizioni, si può confrontare qualsiasi titolo con tutti gli altri calcolando la similarità tra i relativi vettori. Questo approccio consente raccomandazioni basate sul significato del contenuto, indipendentemente dai generi o da altre etichette esplicite.

### Modello ibrido: embedding + metadati

La raccomandazione semantica può essere ulteriormente potenziata integrando anche la rappresentazione strutturale (generi, rating, tipo, regista). Si definisce così una similarità pesata che combina due contributi:

- similarità semantica tra descrizioni
- similarità strutturale basata su feature categoriali

La combinazione avviene tramite una media pesata tra le due componenti, regolabile tramite un parametro che definisce l'importanza relativa del contenuto testuale rispetto alla struttura.

Questo modello ibrido consente di sfruttare punti di forza complementari: la profondità semantica degli embedding e l’informazione esplicita contenuta nei metadati.


In [12]:
!pip install sentence-transformers

In [13]:
from sentence_transformers import SentenceTransformer

# modello = SentenceTransformer('all-MiniLM-L6-v2')  # Veloce e buono
modello = SentenceTransformer('all-distilroberta-v1')

/opt/anaconda3/envs/pycon-beginners-25/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
def genera_embedding_descrizioni(dati, modello):
    descrizioni = [r['description'] if r['description'] else '' for r in dati]
    return modello.encode(descrizioni, show_progress_bar=True)

In [15]:
from numpy import dot
from numpy.linalg import norm

def similarita_coseno_vec(v1, v2):
    if norm(v1) == 0 or norm(v2) == 0:
        return 0.0
    return dot(v1, v2) / (norm(v1) * norm(v2))


In [16]:
def raccomanda_embedding(dati, embeddings, titolo_input, top_n=5):
    titolo_input = titolo_input.strip().lower()
    index = next((i for i, r in enumerate(dati) if r['title'].strip().lower() == titolo_input), None)

    if index is None:
        print("Titolo non trovato.")
        return

    v_input = embeddings[index]
    risultati = []

    for i, v in enumerate(embeddings):
        if i == index:
            continue
        sim = similarita_coseno_vec(v_input, v)
        risultati.append((dati[i]['title'], sim))

    top = sorted(risultati, key=lambda x: x[1], reverse=True)[:top_n]

    print(f"\nTitoli simili a '{dati[index]['title']}' (con embedding semantico):\n")
    for titolo, score in top:
        print(f"{titolo} — similarità: {round(score, 2)}")


In [17]:
embedding_descrizioni = genera_embedding_descrizioni(dati, modello)


NameError: name 'dati' is not defined

In [ ]:
embedding_descrizioni[0].shape

In [ ]:

titolo = input("Inserisci il titolo di un film o serie: ")
raccomanda_embedding(dati, embedding_descrizioni, titolo)

In [ ]:
def raccomanda_embedding_ibrido(dati, embedding_descrizioni, titolo_input, peso_emb=0.7, top_n=5):
    voc_gen = costruisci_vocabolario_generi(dati)
    voc_rat, voc_typ, voc_dir = costruisci_vocabolari_categorici(dati)

    titolo_input = titolo_input.strip().lower()
    index = next((i for i, r in enumerate(dati) if r['title'].strip().lower() == titolo_input), None)

    if index is None:
        print("Titolo non trovato.")
        return

    v_emb_input = embedding_descrizioni[index]
    v_struct_input = vettore_completo(dati[index], voc_gen, voc_rat, voc_typ, voc_dir)

    risultati = []
    for i, r in enumerate(dati):
        if i == index:
            continue

        v_emb = embedding_descrizioni[i]
        v_struct = vettore_completo(r, voc_gen, voc_rat, voc_typ, voc_dir)

        sim_emb = similarita_coseno_vec(v_emb_input, v_emb)
        sim_struct = similarita_coseno(v_struct_input, v_struct)

        sim_finale = peso_emb * sim_emb + (1 - peso_emb) * sim_struct
        risultati.append((r['title'], sim_finale))

    top = sorted(risultati, key=lambda x: x[1], reverse=True)[:top_n]

    print(f"\nTitoli simili a '{dati[index]['title']}' (embedding + colonne):\n")
    for titolo, score in top:
        print(f"{titolo} — similarità: {round(score, 2)}")



In [ ]:
titolo = input("Inserisci un titolo: ")
raccomanda_embedding_ibrido(dati, embedding_descrizioni, titolo, peso_emb=0.4)

## Ultimo step - Profilazione utente e raccomandazione personalizzata

Fino ad ora la raccomandazione era centrata su un titolo fornito in input. Ora passiamo a un'impostazione più realistica: generare raccomandazioni **a partire da una lista di titoli preferiti** da parte dell’utente.

### Costruzione del profilo utente

Un profilo utente è rappresentato da un vettore che sintetizza le preferenze implicite contenute nei film preferiti. Questo avviene tramite:

- **Media degli embedding testuali**: aggregazione semantica delle descrizioni dei titoli scelti
- **Media dei vettori strutturali**: aggregazione binaria di generi, rating, tipo e registi associati

Queste due componenti rappresentano rispettivamente il profilo **semantico** e **strutturale** dell’utente.

### Raccomandazione ibrida basata sul profilo

La funzione di raccomandazione confronta ciascun titolo del dataset con il profilo dell’utente:

- La similarità viene calcolata sia nel **feature space semantico** (embedding) che nel **feature space strutturale**
- Il punteggio finale è una media pesata delle due componenti, regolata da un parametro che bilancia l’influenza del contenuto testuale rispetto ai metadati

Questo approccio permette di generare suggerimenti **personalizzati e diversificati**, andando oltre la semplice similarità con un singolo film.

### Considerazioni

- Il profilo è **dinamico**: ogni variazione nella lista di preferiti cambia la rappresentazione e, di conseguenza, l'intero set raccomandato.
- L’uso della media vettoriale, seppur semplice, si dimostra efficace per catturare la “direzione generale” delle preferenze dell’utente nello spazio semantico.
- È possibile estendere questo modello introducendo pesi sui film (es. valutazioni esplicite) o tecniche più avanzate di aggregazione (es. attention).

Questo modulo conclude la transizione verso un **sistema di raccomandazione completo**, in grado di operare su input testuale libero e adattarsi al profilo dell’utente.



In [ ]:
# INPUT: 
#   film_preferiti (list of str) - titoli scelti dall’utente
#   dati (list of dict) - dataset dei titoli Netflix
#   embedding_descrizioni (list of array) - vettori di embedding semantici per ciascun titolo
# OUTPUT: 
#   array - vettore medio che rappresenta il profilo semantico dell’utente, oppure None se nessun titolo è trovato
# SCOPO: 
#   costruire un vettore di profilo utente mediando gli embedding dei titoli preferiti
def profilo_utente(film_preferiti, dati, embedding_descrizioni):
    pass

# INPUT: 
#   film_preferiti (list of str) - titoli scelti dall’utente
#   dati (list of dict) - dataset dei titoli Netflix
#   embedding_descrizioni (list of array) - embedding semantici per ciascun titolo
#   voc_gen, voc_rat, voc_typ, voc_dir (list of str) - vocabolari strutturali per generi, rating, tipo e registi
# OUTPUT: 
#   tuple (array, list of float) - profilo utente: media degli embedding e media dei vettori strutturali; (None, None) se nessun titolo è valido
# SCOPO: 
#   costruire un profilo utente ibrido combinando rappresentazione semantica e strutturale dei titoli preferiti
def profilo_utente_ibrido(film_preferiti, dati, embedding_descrizioni, voc_gen, voc_rat, voc_typ, voc_dir):
    pass

# INPUT: 
#   dati (list of dict) - dataset dei titoli Netflix
#   embedding_descrizioni (list of array) - embedding semantici per ciascun titolo
#   film_preferiti (list of str) - titoli scelti dall’utente
#   peso_emb (float) - peso assegnato alla componente semantica (default: 0.7)
#   top_n (int) - numero di titoli da raccomandare (default: 5)
# OUTPUT: 
#   None - la funzione stampa i risultati a schermo
# SCOPO: 
#   raccomandare i top-N titoli più affini al profilo utente, combinando embedding e feature strutturate
def raccomanda_ibrido_utente(dati, embedding_descrizioni, film_preferiti, peso_emb=0.7, top_n=5):
    pass



In [ ]:
film_preferiti = [
    "Once upon a time in the west",
    "Dances with Wolves",
    "The Outlaw Josey Wales"
]

# raccomanda_ibrido_utente(dati, embedding_descrizioni, film_preferiti, peso_emb=0.4)